# Measuring a Balmer Decrement from a Spectrum

This tutorial will go over basic python commands, plotting and working with spectra, measuring equivalent widths of emission lines, and finally, using balmer line measurements to infer extinction by dust along the line of sight.  I'm going to roughly follow https://arxiv.org/abs/1109.2597.

In [ ]:
# some basic imports
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

First, let's look at a spectrum.  This will load in an example spectrum of a host galaxy from the Foundation supernova survey (https://ui.adsabs.harvard.edu/abs/2018MNRAS.475..193F/abstract) using a file in the package `SN_site_spectra/` subdirectory.

In [ ]:
wave,flux,fluxerr = np.loadtxt('SN_site_spectra/ATLAS17dzg-combined-20180116_ap6_2.0_kpc_SN.txt',unpack=True)
plt.plot(wave,flux)
plt.xlabel('Wavelength ($\AA)')
plt.ylabel('Flux')

The figure above is a spectrum taken at the location of SN ATLAS17dzg.  Here's what the SN location and host galaxy look like in an image from the Sloan Digital Sky Survey (SDSS):

In [ ]:
from IPython.display import Image
Image("_static/ATLAS17dzg_host.png")

## Properties of the Host Galaxy Spectrum

The host galaxy spectrum above is for a star-forming galaxy.  You can tell because a star-forming galaxy should have lots of gas that gets irradiated by flux from hot O/B stars, producing emission lines.  Let's label a few of these emission lines so you can see what we're looking at:

In [ ]:
## a few lines - Oxygen, Hydrogen
emissionlines = [                 
[3726.15991210938, '[OII]'],
[3728.90991210938, '[OII]'],
[4101.0, 'HDELTA'],
[4340.4677734375, 'HGAMMA'],
[4861.31982421875, 'HBETA'],
[4958.91015625, '[OIII]4959'],
[5006.83984375, '[OIII]5007'],
[6562.81689453125, 'HALPHA'],]

In [ ]:
# let's make the figure a little bigger
plt.rcParams['figure.figsize'] = (16,6)

# re-plot the spectrum
plt.plot(wave,flux,color='k')
plt.xlabel('Wavelength ($\AA)')
plt.ylabel('Flux')
plt.xlim([3000,7000])

# add in the oxygen and hydrogen lines that we might expect
# but, this galaxy is redshifted by the expansion of space, so we have to add in a doppler shift to put the lines
# in the observer frame
z = 0.05217
# I got the above redshift from SDSS, but you can easily measure it by comparing 
# the observed location of H-alpha (the reddest line here) to the expected location, 6562.8 Angstroms
for e in emissionlines:
    plt.axvline(e[0]*(1+z),ls='--')
    plt.text(e[0]*(1+z),0.1,e[1],rotation=90)

## Measuring the Balmer Decrement

To measure the dust at this location in the host galaxy, we have to measure what are called the "equivalent widths" of the H-alpha versus the H-beta lines.  If H-gamma is measureable (it is for this spectrum), it's probably a good idea to measure that too.

The "equivalent width" is the amount of flux contained in an emission line in units of the nearby "continuum" flux (aka, the flux when no emission lines are present).  Here's wikipedia for the official definition: https://en.wikipedia.org/wiki/Equivalent_width.

So to measure this we basically need an estimate of the continuum and a measurement of the integrated line flux.

### The H-beta Line

Looking at H-beta, there is a little bit of absorption in the vicinity probably caused by the spectra of old stars.  Otherwise, the line looks to be in a relatively smooth part of the spectrum.  Let's try and get a line flux.

In [ ]:
# let's work in rest frame now:
restwave = wave/(1+z)
iwave = (restwave > 4800) & (restwave < 4900)
plt.plot(restwave[iwave],flux[iwave])

In [ ]:
# estimate and subtract the nearby continuum with a "cubic spline"
from scipy.interpolate import CubicSpline

# ignoring the region with the line flux
iwave_noline = ((restwave > 4800) & (restwave < 4858)) | ((restwave > 4865) & (restwave < 4900))

cs = CubicSpline(restwave[iwave_noline], flux[iwave_noline])
plt.plot(restwave[iwave],flux[iwave])
plt.plot(restwave[iwave],cs(restwave[iwave]))

In [ ]:
# subtract off the continuum and measure the line flux
iwave_lineonly = (restwave > 4850) & (restwave < 4870)
linesub = flux[iwave_lineonly] - cs(restwave[iwave_lineonly])

# finally, follow Wikipedia to get the equivalen width
ew = np.trapz(linesub/cs(restwave[iwave_lineonly]),x=restwave[iwave_lineonly])
print(f'Equivalent H-beta width: {ew:.3f}')